# Utils

This notebook contains an immutable struct to hold parameter values and primitives functions of the model.

## Define an immutable struct to hold parameters

In [2]:
struct Params
    """
    Immutable struct that contains parameters of the model
    """
    #Economic Parameters
    #-------------------
    alpha::Float64
    beta::Float64
    gamma::Float64
    delta::Float64
    rho::Float64 #persistence of aggregate shock
    a_min::Float64 #borrowing constraint
    a_max::Float64 #maximum capital
    nTrans::Int64 #number of elements in the endoegeneous state transition matrix
    jfp::Float64 # job finding probability
    jlp::Float64 # job losing probability
    exog_trans::Array{Float64,2} #transition matrix
    endowment_low::Float64 #labor low state
    endowment_high::Float64 #labor high state
    Lbar::Float64 #normalization constant
    #Other parameters
    #----------------
    nX::Int64 #number of aggregate variables
    nEps::Int64 #dimension of shock
    nI::Int64 #numer of idiosyncratic states
    iZ::Int64 #index for the value of Z (aggregate productivity)
    iR::Int64 #index for the value of R (interest rate)
    iK::Int64 #index for the value of K (capital)
    iY::Int64 #index for the value of Y (output)
    iC::Int64 #index for the value of C (consumption)
    grid_size::Int64 #Number of points on the grid for savings
    grid_savings::Array{Float64,1} #grid for savings
    grid_savings_long::Array{Float64,1} #grid for savings repeated nI times
    grid_savings_reshaped::Array{Float64,2} #grid for savings reshaped
    grid_endowment::Array{Float64,1} #grid for labor endowments
    grid_endowment_reshaped::Array{Float64,2} #grid for labor endowments
end

# Constructor for type Params
function Params( ; alpha::Float64 = 0.4, beta::Float64 = 0.98, gamma::Float64 = 2.0, 
                    delta::Float64 = 0.02, rho::Float64 = 0.95, a_min::Float64 = 0.0, a_max::Float64 = 200.0,
                    nTrans::Int64 = 2, jfp::Float64 = 0.6, jlp::Float64 = 0.2,
                    endowment_low::Float64 = 1.0, endowment_high::Float64 = 2.5, Lbar::Float64 = 1.0,
                    nX::Int64 = 5, nEps::Int64 = 1, nI::Int64 = 2, grid_size::Int64 = 201,
                    )
    #Indices
    iZ, iR, iK, iY, iC = collect(1:1:nX)
    
    #Exogenous transition matrix. Convention: rows = future state; columns = current state
    exog_trans = [[1.0 - jfp; jfp] [jlp; 1.0 - jlp]]
    
    #Labor endowments:
    grid_endowment = [endowment_low; endowment_high]
    Lbar = dot(grid_endowment, get_stationary_dist(exog_trans))
    grid_endowment = grid_endowment ./ Lbar
    Lbar = 1.0
    
    #grid for savings
    grid_savings = collect(range(a_min^(0.25), stop=a_max^(0.25), length=grid_size)).^4
    
    #repeat grid savings nI times
    grid_savings_long = repeat(grid_savings, nI)
    
    #reshape grid for savings
    grid_savings_reshaped = transpose(repeat(grid_savings, outer=[1, nI]))
    
    #reshape grid of endowments
    grid_endowment_reshaped = repeat(grid_endowment, outer = [1, grid_size])
    
    Params(alpha,
            beta,
            gamma,
            delta,
            rho,
            a_min,
            a_max,
            nTrans,
            jfp,
            jlp,
            exog_trans,
            endowment_low,
            endowment_high,
            Lbar,
            nX, 
            nEps,
            nI,
            iZ,
            iR,
            iK,
            iY,
            iC,
            grid_size,
            grid_savings,
            grid_savings_long,
            grid_savings_reshaped,
            grid_endowment,
            grid_endowment_reshaped)
end

Params

## Search nearest value on a grid

In [ ]:
# Function to find the index corresponding to the closest value on a grid:
# Source: https://discourse.julialang.org/t/findnearest-function/4143/4
function searchsortednearest(a::Array{Float64,1},x::Float64)
    """
    Returns the index of the closest value to x on grid a
    a::Array{Float64,1}: grid
    x::Float64: value to be found
    """
    idx = searchsortedfirst(a,x)
    if (idx==1); return idx; end
    if (idx>length(a)); return length(a); end
    if (a[idx]==x); return idx; end
    if (abs(a[idx]-x) < abs(a[idx-1]-x))
      return idx
    else
      return idx-1
    end
end

## Calculate stationary distribution

In [5]:
function get_stationary_dist(T::Array{Float64,2})
    """
    Returns the stationary distribution
    of A'=TA
    """
    F = eigen(T) #F.values and F.vectors
    #Find index eigenvalue that is equal to 1
    eigenvalue_real = real(F.values)
    i = searchsortednearest(eigenvalue_real, 1.0)
    #Select the corresponding eigenvector and normalize
    eigenvector_real = real(F.vectors[:,i])
    return eigenvector_real/sum(eigenvector_real)
    
end

get_stationary_dist (generic function with 1 method)

## Utility and marginal utility functions

In [3]:
function u(c::Array{Float64,1}, p::Params)
    """
    Utility function when the input is an array
    """
    return (c.^(1.0-p.gamma)-1.0)./(1.0-p.gamma)
end


#Marginal utility function
function u′(c::Array{Float64,1}, p::Params)
    """
    Marginal utility function
    """
    return c.^(-p.gamma)
end


#Marginal utility function
function u′!(up::Array{Float64,2}, c::Array{Float64,2}, p::Params)
    """
    Marginal utility function, mutates the input up
    """
    up[:,:] = c.^(-p.gamma)
end


function u′_inv(c::Array{Float64,1}, p::Params)
    """
    #Inverse marginal utility function
    """
    return c.^(-1.0/p.gamma)
end

function u′_inv!(up_inv::Array{Float64,2}, c::Array{Float64,2}, p::Params)
    """
    #Inverse marginal utility function, mutates the input up_inv
    """
    up_inv[:,:] = c.^(-1.0/p.gamma)
end

u′_inv! (generic function with 1 method)

## Interest rate and wage

In [1]:
function R(K::Float64, Z::Float64, p::Params)
    """
    Interest rate
    K: level of capital
    Z: aggregate state
    """
    return Z * p.alpha*(K/p.Lbar)^(p.alpha - 1.0) + 1.0 - p.delta
end

function W(K::Float64, Z::Float64, p::Params)
    """
    Wage
    K: level of capital
    Z: aggregate state
    """
    return Z*(1.0 - p.alpha)*(K/p.Lbar)^(1.0 - p.alpha)
end

UndefVarError: UndefVarError: Params not defined